# **Question 2**

In [1]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.


* The idx2numpy package is specifically designed for working with data in the IDX file format, which is commonly used to store large datasets in machine learning

In [2]:
!pip install idx2numpy

  Preparing metadata (setup.py) ... done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=2a448140e07d6abf44b3f5a61e2520444d81a90ab9073fbae25fe0307fc5a2a3
  Stored in directory: /root/.cache/pip/wheels/e0/f4/e7/643fc5f932ec2ff92997f43f007660feb23f948aa8486f1107
Successfully built idx2numpy


In [56]:
# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models import ResNet18_Weights
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [57]:
# Chemins vers les fichiers IDX du dataset MNIST (images et labels pour entraînement et test).
train_images_path = "/kaggle/input/datasetmnist/train-images.idx3-ubyte"
train_labels_path = "/kaggle/input/datasetmnist/train-labels.idx1-ubyte"
test_images_path = "/kaggle/input/datasetmnist/t10k-images.idx3-ubyte"
test_labels_path = "/kaggle/input/datasetmnist/t10k-labels.idx1-ubyte"

In [58]:
# Normaliser les pixels des images en les mettant dans la plage [0, 1].
def normalize_images(images):
    return images / 255.0

In [59]:
# Charger les images et labels d'entraînement et de test depuis les fichiers IDX en tableaux NumPy.
train_images = idx2numpy.convert_from_file(train_images_path)
train_labels = idx2numpy.convert_from_file(train_labels_path)
test_images = idx2numpy.convert_from_file(test_images_path)
test_labels = idx2numpy.convert_from_file(test_labels_path)

In [60]:
# Define a Dataset for MNIST with bounding boxes
class MNISTFasterRCNNDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Simulated bounding box
        boxes = torch.tensor([[5, 5, 23, 23]], dtype=torch.float32)
        labels = torch.tensor([label], dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel

        return image, target

In [61]:
# Create datasets and DataLoaders
train_dataset = MNISTFasterRCNNDataset(train_images, train_labels)
test_dataset = MNISTFasterRCNNDataset(test_images, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [62]:
# Load Faster R-CNN model with ResNet-18 backbone
backbone = resnet_fpn_backbone('resnet18', weights=ResNet18_Weights.IMAGENET1K_V1)
model = FasterRCNN(backbone, num_classes=10)

In [63]:
# Modify predictor head for MNIST (10 classes)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=10)

In [65]:
# Define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

In [66]:
# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride

In [67]:
# Function to train for one epoch
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    epoch_loss = 0
    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()
    avg_loss = epoch_loss / len(data_loader)
    print(f"Epoch {epoch}, Loss: {avg_loss}")
    return avg_loss

In [71]:
torch.cuda.empty_cache()

In [ ]:
# Train and record losses
num_epochs = 2
epoch_losses = []

for epoch in range(num_epochs):
    loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    epoch_losses.append(loss)

# Plot epoch losses
plt.figure(figsize=(8, 6))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.show()

In [40]:
# Evaluate model
from sklearn.metrics import accuracy_score, f1_score

In [41]:
def evaluate_model(model, test_loader):
    model.eval()
    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for images, targets in test_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for output, target in zip(outputs, targets):
                if len(output["labels"]) > 0:
                    predicted_labels.append(output["labels"][0].item())
                else:
                    predicted_labels.append(-1)
                true_labels.append(target["labels"].item())

    valid_indices = [i for i, label in enumerate(predicted_labels) if label != -1]
    predicted_labels = [predicted_labels[i] for i in valid_indices]
    true_labels = [true_labels[i] for i in valid_indices]

    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average="weighted")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return true_labels, predicted_labels

# Get evaluation metrics
true_labels, predicted_labels = evaluate_model(model, test_loader)

# Plot confusion matrix
cm = confusion_matrix(true_labels, predicted_labels, labels=range(10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap="Blues", xticks_rotation="vertical")
plt.title("Confusion Matrix")
plt.show()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride